# Exercício 3.2 - Cilindro

> Nome: Michelle Wingter da Silva
>
> NUSP: 10783243

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Esfera", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4). Use ela como matriz de transformação, resultante de uma sequências de outras transformações (e.g. rotação + translação)
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação.

In [3]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando um Cilindro



In [11]:
import math

PI = 3.141592
r = 0.2 # raio
num_sectors = 30 # qtd de sectors (longitude)
num_stacks = 10 # qtd de stacks (latitude)

# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=0.9/num_stacks # variar de 0 até 2

# Entrada: angulo t, raio r, altura h
# Saida: coordenadas no cilindro
def F(t,h,r):
    x = r*math.cos(t)
    y = r*math.sin(t)
    z = h
    return (x,y,z)
    

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_list = []
for i in range(0,num_sectors): # para cada sector (longitude)
    
    for j in range(0,num_stacks): # para cada stack (latitude)
         
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if (i+1==num_sectors):
            un = PI*2
        else: 
            un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        vn = (j+1)*stack_step
        
        # verticies do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)


total_vertices = num_sectors * num_stacks * 6#1800
vertices = np.zeros(total_vertices+(num_sectors*2), [("position", np.float32, 3)])

angle = 0.0
z = 0
for counter in range(total_vertices, total_vertices+num_sectors):
    angle += 2*PI/29
    vertices_list.append(F(angle,z,r))    
    
    
angle = 0.0
z = 0.9
for counter in range(total_vertices+num_sectors, total_vertices+(num_sectors*2)):
    angle += 2*PI/29
    vertices_list.append(F(angle,z,r))

vertices["position"] = np.array(vertices_list)


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [12]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [13]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [14]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [15]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [16]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [17]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, nós exibimos a janela!


In [18]:
glfw.show_window(window)

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [19]:
# exemplo para matriz de translacao
t_x = 0
t_y = 0
t_z = 0
m_x = 1
m_y = 1
m_z = 1

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, t_z, d, e
    
#     print('[key event] key=',key)
#     print('[key event] scancode=',scancode)
#     print('[key event] action=',action)
#     print('[key event] mods=',mods)
#     print('-------')
    
    if key == 262: t_x += 0.01 #seta direita
    if key == 263: t_x -= 0.01 #seta esquerda
        
    if key == 265: t_y += 0.01 #seta cima
    if key == 264: t_y -= 0.01 #seta baixo
        
    if (key == 32):  d+=0.1   #espaço       
    if (key == 342): e-=0.1   #alt esquerdo

def mouse_button_callback(window,button,action,mods):
    global m_x, m_y, m_z
    if (button == 1):
        m_x *= 2
        m_y *= 2
        m_z *= 2
    if (button == 0):
        m_x *= 0.5
        m_y *= 0.5
        m_z *= 0.5

    
glfw.set_mouse_button_callback(window,mouse_button_callback)
glfw.set_key_callback(window,key_event)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE para modelar os triângulos (que formarão outros polígonos) da superfície da nossa esfera.

In [20]:
import math
import random
d = 0.0
e = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    ### apenas para visualizarmos a esfera rotacionando
    #d -= 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    cos_e = math.cos(e)
    sin_e = math.sin(e)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #ROTAÇÃO
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_e,  0.0, sin_e, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_e, 0.0, cos_e, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    #ESCALA
    mat_scale = np.array([          m_x, 0.0, 0.0, 0.0, 
                                    0.0, m_y, 0.0, 0.0, 
                                    0.0, 0.0, m_z, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    #TRANSLAÇÃO
    mat_translation = np.array([    1.0,   0.0,    0.0, t_x, 
                                    0.0,   1.0,    0.0, t_y, 
                                    0.0,   0.0,    1.0, t_z, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    #IDENTIDADE
    mat_identidade = np.array([     1.0,  0.0, 0.0, 0.0, 
                                    0.0,  1.0, 0.0, 0.0, 
                                    0.0,  0.0, 1.0, 0.0, 
                                    0.0,  0.0, 0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
 
    for triangle in range(0,len(vertices)-(num_sectors*2),3):     
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()  
        glUniform4f(loc_color, R, G, B, 1.0) # randomizando as cores de cada triangulo
        glDrawArrays(GL_TRIANGLES, triangle, 3)     

        
    
    glUniform4f(loc_color, 0, 0, 1, 1.0) ### azul
    glDrawArrays(GL_TRIANGLE_FAN, len(vertices)-(num_sectors*2), 30)     
    glDrawArrays(GL_TRIANGLE_FAN, len(vertices)-num_sectors, 30)    

    
    glfw.swap_buffers(window)

glfw.terminate()